In [5]:
from dotenv import load_dotenv
import os
from llama_index.llms.anthropic import Anthropic
from llama_index.core.tools import FunctionTool
import nest_asyncio

In [14]:
# Load environment variables from .env file
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
CLAUDE_API_KEY = os.getenv('CLAUDE_API_KEY')
LLAMA_CLOUD_API_KEY=os.getenv('LLAMA_CLOUD_API_KEY')

In [15]:

nest_asyncio.apply()

import openai
openai.api_key = OPENAI_API_KEY


In [19]:
path_pdf = 'C:/Projects/'
output_directory_path = path_pdf+'test_claude'

In [22]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_parse import LlamaParse
pdf_file = path_pdf + 'citi202310k.pdf'
# set up parser
parser = LlamaParse(api_key=LLAMA_CLOUD_API_KEY, result_type="markdown")
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_files=[pdf_file], file_extractor=file_extractor).load_data()
# Index the documents in a vectorStore
index_citi = VectorStoreIndex.from_documents(documents)

Started parsing the file under job_id 441d1d54-e752-4337-bde6-f6866f090575


In [23]:
documents

[Document(id_='2019365c-11fe-4642-914f-3c3c4d0a1411', embedding=None, metadata={'file_path': 'C:\\Projects\\citi202310k.pdf', 'file_name': 'citi202310k.pdf', 'file_type': 'application/pdf', 'file_size': 3548798, 'creation_date': '2024-05-02', 'last_modified_date': '2024-05-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='## SECURITIES AND EXCHANGE COMMISSION UNITED STATES WASHINGTON, D.C. 20549\n\n(Mark One) FORM 10-K\n\n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2023 OR ☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from to Commission file number 1-9924 Citigroup Inc. (Exact name of registrant as s

In [25]:
from llama_index.llms.openai import OpenAI
llm_openai = OpenAI(model="gpt-4-turbo", temperature=0.1)
query_engine_citi = index_citi.as_query_engine(llm=llm_openai)
# query the engine
query = "What was the Net Income in 2023, the Reported one?"
response_citi = query_engine_citi.query(query)
print("*"*50)
3
print(f"The answer to the question '''{query}''' is :\n")
response_citi.response

**************************************************
The answer to the question '''What was the Net Income in 2023, the Reported one?''' is :



'The net income for 2023 was $2,107 million.'

In [27]:
print(response_citi.response)

The net income for 2023 was $2,107 million.


In [29]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
#Create the Query Engine Tool over the RAG pipeline
query_engine_tool = QueryEngineTool(
query_engine=query_engine_citi,
metadata=ToolMetadata(
name="citi202310k",
description=(
"Provides Annual Report information about Citi for 2023. "
"Use a detailed plain text question as input to the tool."
),
),
)

In [31]:
from llama_index.core.agent import FunctionCallingAgentWorker
llm_anthropic = Anthropic(model="claude-3-opus-20240229", api_key=CLAUDE_API_KEY)
agent_worker = FunctionCallingAgentWorker.from_tools(
[query_engine_tool], llm=llm_anthropic, verbose=True
)
agent = agent_worker.as_agent()

In [32]:
query = "What was the Net Income in 2023, the Reported one?"
response = agent.chat(query)
print(str(response))

Added user message to memory: What was the Net Income in 2023, the Reported one?
=== LLM Response ===
<thinking>
The relevant tool to answer this question is citi202310k, which provides information from Citi's 2023 Annual Report. The tool takes a single "input" parameter, which should be a detailed plain text question.

The user has asked a specific question about Citi's Net Income in 2023, which should be answerable from the 2023 Annual Report. No additional information is needed to formulate the query to the tool.
</thinking>
=== Calling Function ===
Calling function: citi202310k with args: {"input": "What was Citi's reported Net Income in 2023?"}
=== Function Output ===
Citi's reported net income in 2023 was $9,228 million.
=== LLM Response ===
Based on the information from Citi's 2023 Annual Report, Citi's reported Net Income in 2023 was $9,228 million.
assistant: Based on the information from Citi's 2023 Annual Report, Citi's reported Net Income in 2023 was $9,228 million.


In [34]:
query = "What was the evolution of the Net Income from 2022 to 2023, the Reported one. The document has data for 2022 and 2023?"
response = agent.chat(query)
print(str(response))

Added user message to memory: What was the evolution of the Net Income from 2022 to 2023, the Reported one. The document has data for 2022 and 2023?
=== LLM Response ===
<thinking>
You're right, I apologize for my previous response. Let me check again if the 2023 Annual Report also contains 2022 data that can be used to answer the question about the change in Net Income.
</thinking>
=== Calling Function ===
Calling function: citi202310k with args: {"input": "What were Citi's reported Net Income amounts for 2022 and 2023, and how did Net Income change from 2022 to 2023?"}
=== Function Output ===
Citi's reported net income for 2022 was $14,845 million and for 2023 was $9,228 million. The net income decreased by 38% from 2022 to 2023.
=== LLM Response ===
Based on the information from Citi's 2023 Annual Report:

Citi's reported Net Income was:
2022: $14,845 million 
2023: $9,228 million

Net Income decreased from $14,845 million in 2022 to $9,228 million in 2023, a decline of 38%.
assista

In [35]:
query = "What was the evolution in term of percentage of the Net Income from 2022 to 2023, the Reported one?"
response = agent.chat(query)
print(str(response))


Added user message to memory: What was the evolution in term of percentage of the Net Income from 2022 to 2023, the Reported one?
=== LLM Response ===
<thinking>
The previous response already provided the percentage change in Citi's reported Net Income from 2022 to 2023. Net Income decreased by 38% from $14,845 million in 2022 to $9,228 million in 2023.

To confirm, I will check the annual report data again.
</thinking>
=== Calling Function ===
Calling function: citi202310k with args: {"input": "By what percentage did Citi's reported Net Income change from 2022 to 2023?"}
=== Function Output ===
Citi's reported net income decreased by 38% from 2022 to 2023.
=== LLM Response ===
Citi's 2023 Annual Report shows that the company's reported Net Income decreased by 38% from 2022 to 2023.
assistant: Citi's 2023 Annual Report shows that the company's reported Net Income decreased by 38% from 2022 to 2023.


In [38]:
def evolution(a: float, b: float) -> float:
    """Evolution in percentage from value b to value a"""
    return f"{round(100*(a/b-1),0)}%"
evolution_tool = FunctionTool.from_defaults(fn=evolution)

In [39]:
def cagr(a: float, b: float, n: int) -> float:
    """Compound annual growth rate CAGR from value b to value a over n period"""
    return f"{round(100*((a/b)^(1/n)-1),0)}%"
cagr_tool = FunctionTool.from_defaults(fn=cagr)


In [40]:
def price_earning_ratio(price: float, eps: float) -> float:
    """Price per earning ratio is a ratio of the share price devided by the earning per share(EPS) value"""
    return price/eps
pe_tool = FunctionTool.from_defaults(fn=price_earning_ratio)

In [42]:
agent_worker2 = FunctionCallingAgentWorker.from_tools(
[query_engine_tool, evolution_tool], llm=llm_anthropic, verbose=True
)
agent2 = agent_worker2.as_agent()
query = "What was the evolution of the Net Income from 2022 to 2023, the Reported one?"
response = agent2.chat(query)
print(str(response))

Added user message to memory: What was the evolution of the Net Income from 2022 to 2023, the Reported one?
=== LLM Response ===
<thinking>
To calculate the evolution of Citi's Net Income from 2022 to 2023, I need two pieces of information:
1. Citi's Net Income for 2022 
2. Citi's Net Income for 2023
The citi202310k tool can likely provide this information from Citi's 2023 annual report. I'll ask it for the specific Net Income numbers.
</thinking>
=== Calling Function ===
Calling function: citi202310k with args: {"input": "What was Citi's reported Net Income in millions of dollars for 2022 and 2023?"}
=== Function Output ===
Citi's reported net income was $14,845 million in 2022 and $9,228 million in 2023.
=== LLM Response ===
<thinking>
The tool provided the two Net Income values I need:
a (2023 value): $9,228 million 
b (2022 value): $14,845 million
I have the required parameters to calculate the percentage evolution using the evolution tool.
</thinking>
=== Calling Function ===
Call

In [43]:
query = "Compare the evolution of the Net Income between 2021 to 2022 and 2022 to 2023, the Reported one."
response = agent2.chat(query)
print(str(response))

Added user message to memory: Compare the evolution of the Net Income between 2021 to 2022 and 2022 to 2023, the Reported one.
=== LLM Response ===
<thinking>
To compare the evolution of Citi's Net Income between the periods 2021 to 2022 and 2022 to 2023, I need three data points:

1. Citi's Net Income for 2021
2. Citi's Net Income for 2022 
3. Citi's Net Income for 2023

I already have the 2022 and 2023 values from the previous query. I just need the 2021 value to calculate both evolutions and compare them. I'll ask the citi202310k tool for Citi's 2021 Net Income.
</thinking>
=== Calling Function ===
Calling function: citi202310k with args: {"input": "What was Citi's reported Net Income in millions of dollars for 2021?"}
=== Function Output ===
Citi's reported net income for 2021 was $21,952 million.
=== LLM Response ===
<thinking>
Now I have the Net Income values for all three years:

2021: $21,952 million
2022: $14,845 million 
2023: $9,228 million

I can calculate the evolution fro

In [45]:
agent_worker3 = FunctionCallingAgentWorker.from_tools(
[query_engine_tool, evolution_tool, cagr_tool], llm=llm_anthropic, verbose=True
)
agent3 = agent_worker3.as_agent()
query = "What was the compound annual growth of the Net Income from 2021 to 2023, the Reported one?"
response = agent3.chat(query)
print(str(response))

Added user message to memory: What was the compound annual growth of the Net Income from 2021 to 2023, the Reported one?
=== LLM Response ===
<thinking>
To calculate the compound annual growth rate (CAGR) of Citi's Net Income from 2021 to 2023, I need 3 pieces of information:
1. Citi's Net Income in 2023 (a)
2. Citi's Net Income in 2021 (b) 
3. The number of years between 2021 and 2023 (n)

I can use the citi202310k tool to try to find Citi's Net Income values for 2023 and 2021.
</thinking>
=== Calling Function ===
Calling function: citi202310k with args: {"input": "What was Citi's reported Net Income in 2023?"}
=== Function Output ===
Citi's reported net income in 2023 was $9,228 million.
=== Calling Function ===
Calling function: citi202310k with args: {"input": "What was Citi's reported Net Income in 2021?"}
=== Function Output ===
Citi's reported net income in 2021 was $21,952 million.
=== LLM Response ===
<thinking>
Great, I now have the 3 pieces of information needed to calculate